<a href="https://colab.research.google.com/github/davidwhogg/BadForScience/blob/main/notebooks/bias_in_regression_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biases in regression outputs
The goal of this notebook is to show that when the data have small information about the label, then population-level inferences using the output labels are biased.

## Author:
- **David W Hogg** (NYU) (MPIA) (Flatiron)

## License:
Copyright 2023, 2024 the author. All code is released open-source under the MIT License.

## To-do:
- Make plots that are publication-worthy.
- Play with hyper-parameters of the MLP and the data generation to see how the results depend on choices.

In [ ]:
import numpy as np
import pylab as plt
from sklearn.neural_network import MLPRegressor
rng = np.random.default_rng(17)

In [ ]:
# set hyper-parameters
M = 110 # dimension of the data (thinking XP spectra)
K = 13 # number of latent (unknown) parameters beyond age and guiding radius
N_train = 2 ** 12
N_valid = 2 ** 11
N_extra = 2 ** 17
N_total = N_train + N_valid + N_extra
maxR = 14.0 # kpc
maxage = 13.0 # Gyr
agerange = 3.0 # Gyr
agefactor = 0.08 # make this smaller to reduce age information in the data set
datanoise = 0.05 # noise level for the data
agenoise = 1.0 # Gyr age label noise level
print(N_train, N_valid, N_total, maxR)

In [ ]:
# make latent true ages and other properties
true_guide_radii = rng.uniform(0., maxR, size=N_total)
true_ages = maxage - (maxage / maxR) * true_guide_radii + agerange * rng.normal(size=N_total)
okay = (true_ages > 0.) * (true_ages < maxage)
true_guide_radii = true_guide_radii[okay]
true_ages = true_ages[okay]
N_total = len(true_ages)
N_test = N_total - N_train - N_valid
true_latents = np.random.normal(size=(N_total, K))
print(true_guide_radii.shape, true_ages.shape, true_latents.shape)

In [ ]:
# make true data
radius_vec = np.random.normal(size=M)
age_vec = agefactor * np.random.normal(size=M)
latent_vecs = np.random.normal(size=(K, M))
true_data = (true_ages[:, None] * age_vec[None, :]
            + true_latents @ latent_vecs) / np.sqrt(K + 1)
true_data = 1. + np.clip(true_data, -1.0, 0.0) # apply RELU-like nonlinearity
print(true_data.shape)

In [ ]:
# add noise to the data and labels
data = true_data + datanoise * rng.normal(size=true_data.shape)
ages = true_ages + agenoise * rng.normal(size=true_ages.shape)
print(data.shape, ages.shape)

In [ ]:
plt.title("data examples (vertically offset)")
for i in range(10):
    plt.plot(data[i] + 1.2 * i, "k-")
    plt.text(M, 1.2 * i + 0.5, "{:4.1f}".format(ages[i]))
plt.xlim(-0.5, M - 0.5)

In [ ]:
# Make function to take means in bins of data
def means_in_bins(xs, ys):
    nbin = 14
    meanx = np.zeros(nbin)
    meany = np.zeros(nbin)
    stdy = np.zeros(nbin)
    for xmin in range(nbin):
        I = (xs > xmin) * (xs < (xmin + 1.))
        meanx[xmin] = np.mean(xs[I])
        meany[xmin] = np.mean(ys[I])
        stdy[xmin] = np.std(ys[I]) / np.sqrt(np.sum(I))
    return meanx, meany, stdy

In [ ]:
plt.title("full data set")
mean_radii, mean_ages, std_ages = means_in_bins(true_guide_radii, ages)
plt.scatter(true_guide_radii, ages, s=0.2, c="k", alpha=0.25)
plt.plot(mean_radii, mean_ages, "wo", mew=4)
plt.plot(mean_radii, mean_ages, "ro")
radiuslim = (0., 14.)
plt.xlim(radiuslim)
agelim = (0., 15.)
plt.ylim(agelim)
plt.xlabel("guiding radius")
plt.ylabel("measured age")

In [ ]:
# Set up train, validate, and test sets
X_train = data[:N_train]
Y_train = ages[:N_train]
X_valid = data[N_train:N_train + N_valid]
Y_valid = ages[N_train:N_train + N_valid]
X_test = data[N_train + N_valid:]
Y_test = ages[N_train + N_valid:]
print(X_train.shape, X_valid.shape, X_test.shape)

In [ ]:
plt.title("training data (N = {:d})".format(N_train))
radii_train = true_guide_radii[:N_train]
mean_train_radii, mean_train_ages, std_train_ages = means_in_bins(radii_train, Y_train)
plt.scatter(radii_train, Y_train, s=0.2, c="k", alpha=0.8)
shadow_alpha=0.8
plt.plot(mean_radii, mean_ages, "wo", mew=4, alpha=shadow_alpha)
plt.plot(mean_train_radii, mean_train_ages, "ws", mfc="none", mew=4, alpha=shadow_alpha)
plt.plot(mean_radii, mean_ages, "ro", label="true relationship")
plt.plot(mean_train_radii, mean_train_ages, "ks", mfc="none", label="means of training labels in bins")
plt.xlim(radiuslim)
plt.ylim(agelim)
plt.legend()
plt.xlabel("guiding radius")
plt.ylabel("measured age")

In [ ]:
# set up a MLP to solve this problem
MLP_kwargs = {'hidden_layer_sizes': (30,30,30),
              'activation': 'relu',
              'solver': 'adam',
              'alpha': 0.0001,
              'batch_size': 'auto',
              'learning_rate': 'constant',
              'learning_rate_init': 0.001,
              'power_t': 0.5,
              'max_iter': 400,
              'shuffle': True,
              'random_state': None,
              'tol': 0.0001,
              'verbose': True,
              'warm_start': False,
              'momentum': 0.9,
              'nesterovs_momentum': True,
              'early_stopping': False,
              'validation_fraction': 0.1,
              'beta_1': 0.9,
              'beta_2': 0.999,
              'epsilon': 1e-08,
              'n_iter_no_change': 10,
              'max_fun': 15000}
mlp = MLPRegressor(**MLP_kwargs)

In [ ]:
# perform regression
regr = mlp.fit(X_train, Y_train)
print(regr)

In [ ]:
# Check that the regression is okay
Y_valid_hat = regr.predict(X_valid)
print(Y_valid, Y_valid - Y_valid_hat)
print("bias:", np.mean(Y_valid - Y_valid_hat))
print("rms:", np.sqrt(np.mean((Y_valid - Y_valid_hat) ** 2)))

In [ ]:
plt.title("validation set (N = {:d})".format(N_valid))
plt.scatter(Y_valid, Y_valid_hat, s=0.2, c="k")
foo = plt.ylim()
plt.plot(foo, foo, "k-", alpha=0.8, zorder=-10)
plt.ylim(agelim)
plt.xlim(agelim)
plt.xlabel("measured age")
plt.ylabel("regression-predicted age")

In [ ]:
# run on the test set
Y_test_hat = regr.predict(X_test)
print(Y_test_hat.shape)

In [ ]:
plt.title("test set (N = {:d})".format(N_test))
radii_test = true_guide_radii[N_train + N_valid:]
print(radii_test.shape, Y_test_hat.shape)
mean_regr_radii, mean_regr_ages, std_regr_ages = means_in_bins(radii_test, Y_test_hat)
nsigma = [" {:4.1f}".format(ns) for ns in np.abs(mean_regr_ages - mean_ages) / std_regr_ages]
plt.scatter(radii_test, Y_test_hat, s=0.2, c="k", alpha=0.25)
i = 12
plt.text(mean_regr_radii[i], mean_regr_ages[i], nsigma[i], c="w", fontweight=1000, rotation=45)
shadow_alpha=0.8
plt.plot(mean_radii, mean_ages, "wo", mew=4, alpha=shadow_alpha)
plt.plot(mean_train_radii, mean_train_ages, "ws", mfc="none", mew=4, alpha=shadow_alpha)
plt.plot(mean_regr_radii, mean_regr_ages, "wx", mfc="none", mew=4, ms=8, alpha=shadow_alpha)
plt.plot(mean_radii, mean_ages, "ro", label="true relationship")
plt.plot(mean_train_radii, mean_train_ages, "ks", mfc="none", label="means of training labels in bins")
plt.plot(mean_regr_radii, mean_regr_ages, "kx", mfc="none", label="means of regression-predicted labels")
plt.xlim(radiuslim)
plt.ylim(agelim)
plt.legend()
plt.xlabel("guiding radius")
plt.ylabel("regression-predicted age")